<a href="https://colab.research.google.com/github/MASSIMOQSELLA/MyAKarpathyMicrogradTest/blob/main/Sigla_NP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
import sys
from sklearn.model_selection import train_test_split
import time
from datetime import datetime

#!git clone https://github.com/MASSIMOQSELLA/MyAKarpathyMicrogradTest.git

def input_set(true_vectors_path, false_vectors_path): # carica i dati di input dal repository e li restituisce come immagini

  # Caricamento dei dati
  real_vectors = np.load(true_vectors_path)  # Firme vere
  false_vectors = np.load(false_vectors_path)  # Firme false
  print(f"True vectors shape: {real_vectors.shape}")  # Dovrebbe essere (438, 3136)
  print(f"False vectors shape: {false_vectors.shape}")  # Dovrebbe essere (528, 3136)

  # Creazione delle etichette
  real_labels = np.ones((real_vectors.shape[0],1))  # Etichette per le firme vere (1)
  false_labels = np.zeros((false_vectors.shape[0],1))  # Etichette per le firme false (0)
  # Concatenazione dei dati
  data = np.vstack((real_vectors, false_vectors))  # Combina i vettori
  labels = np.vstack((real_labels, false_labels))  # Combina le etichette
  # Shuffle dei dati
  indices = np.arange(data.shape[0])
  np.random.shuffle(indices)
  data = data[indices]
  labels = labels[indices]
  print(f"Dataset pronto: {data.shape}, Labels: {labels.shape}")
  # Divisione train/test
  train_data, test_data, train_labels, test_labels = train_test_split(
      data, labels, test_size=0.2, random_state=42
  )
  # conteggio false vs vere all'interno del train_set e del test_set
  train_false_num = np.count_nonzero(train_labels == 0)
  train_real_num = np.count_nonzero(train_labels == 1)
  test_false_num = np.count_nonzero(test_labels == 0)
  test_real_num = np.count_nonzero(test_labels == 1)
  print(f"Train set> data: {train_data.shape}, labels: {train_labels.shape}, True: {train_real_num}, False: {train_false_num} ")
  print(f"Test set> data: {test_data.shape}, labels: {test_labels.shape}, True: {test_real_num}, False: {test_false_num} ")

  return train_data, test_data, train_labels, test_labels

class SimpleNN:

    def __init__(self, input_size, hidden_size, output_size, learning_rate):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.learning_rate = learning_rate
        # Inizializza i pesi e i bias con He per ReLU
        #self.W1 = np.random.randn(input_size, hidden_size) * 0.01
        self.W1 = np.random.randn(input_size, hidden_size) * np.sqrt(2. / input_size)
        #self.b1 = np.zeros((1, hidden_size))
        self.b1 = np.ones((1, hidden_size)) * 0.1
        self.W2 = np.random.randn(hidden_size, output_size) * np.sqrt(2. / hidden_size)
        self.b2 = np.zeros((1, output_size))
        #self.b2 = np.ones((1, output_size)) * 0.001

    def params(self):
        #Restituisce i parametri (pesi e bias) del modello come un dizionario
        return {
            'W1': self.W1,
            'b1': self.b1,
            'W2': self.W2,
            'b2': self.b2
        }

    def relu(self, Z):
        return np.maximum(0, Z)

    def relu_derivative(self, Z):
      return Z > 0  # Restituisce un array booleano (True/False) che diventa 1/0

    def sigmoid(self, Z):
        return 1 / (1 + np.exp(-Z))

    def sigmoid_derivative(self, Z):
        return self.sigmoid(Z) * (1 - self.sigmoid(Z))

    def forward(self, X):
        # Passo forward
        self.Z1 = np.dot(X, self.W1) + self.b1
        self.A1 = self.relu(self.Z1)
        self.Z2 = np.dot(self.A1, self.W2) + self.b2
        self.A2 = self.sigmoid(self.Z2)
        return self.A2

    def compute_loss(self, X, y):
        #Calcola la funzione di loss utilizzando la Binary Cross-Entropy (BCE)
        predictions = self.forward(X)
        # Aggiungi un epsilon per evitare log(0) e garantire stabilità numerica
        epsilon = 1e-10
        predictions = np.clip(predictions, epsilon, 1 - epsilon)  # Restringi i valori tra [epsilon, 1 - epsilon]
        # Calcola la BCE
        loss = -np.mean(y * np.log(predictions) + (1 - y) * np.log(1 - predictions))
        return loss

    def backward(self, X, y):

        # Calcolare l'errore (output - vero valore)
        m = X.shape[0]  # Numero di esempi nel batch

        # Derivata della loss (BCE) rispetto all'output
        dA2 = self.A2 - y

        # Derivata della perdita rispetto ai pesi e bias dell'output
        dZ2 = dA2 * self.sigmoid_derivative(self.Z2)
        dW2 = np.dot(self.A1.T, dZ2) / m
        db2 = np.sum(dZ2, axis=0, keepdims=True) / m

        # Derivata della perdita rispetto ai pesi e bias del layer nascosto
        dA1 = np.dot(dZ2, self.W2.T)
        dZ1 = dA1 * self.relu_derivative(self.Z1)  # Derivata della ReLU
        dW1 = np.dot(X.T, dZ1) / m
        db1 = np.sum(dZ1, axis=0, keepdims=True) / m

        # Restituisce i gradienti come dizionario
        gradients = {
            'W1': dW1,
            'b1': db1,
            'W2': dW2,
            'b2': db2
        }

        return gradients

    def train(self, X_train, y_train, epochs=1000, reshuffle_int = 200):
        for epoch in range(epochs):
            # Passo in avanti (forward pass)
            #predictions = self.forward(X_train)

            # Calcolare e stampare la perdita (opzionale)
            #loss = np.mean((predictions - y_train) ** 2)
            loss = self.compute_loss(X_train, y_train)

            # Passo indietro (backward pass)
            gradients = self.backward(X_train, y_train)

            # Aggiorna i pesi e i bias con la discesa del gradiente
            self.W1 -= self.learning_rate * gradients['W1']
            self.b1 -= self.learning_rate * gradients['b1']
            self.W2 -= self.learning_rate * gradients['W2']
            self.b2 -= self.learning_rate * gradients['b2']

            if epoch % 100 == 0:
                print(f"Epoch {epoch}/{epochs}, Loss: {loss}")
                # Controllo dei gradienti numerici
                #check_gradients(self, X_train, y_train)

            if (epoch % reshuffle_int == 0) and (epoch != 0):
                print(f"Reshuffling data at epoch {epoch}")
                indices = np.arange(X_train.shape[0])
                np.random.shuffle(indices)
                X_train = X_train[indices]
                y_train = y_train[indices]


def numerical_gradient(X, y, model, epsilon=1e-5):
    """
    Calcola il gradiente numerico della loss rispetto ai pesi del modello
    usando il metodo delle differenze finite.

    X: dati di input
    y: etichette vere
    model: il modello di rete neurale
    epsilon: piccolo valore per le differenze finite
    """
    gradients = {}

    # Calcola la loss originale
    #loss_original = np.mean((model.forward(X) - y) ** 2)
    loss_original = model.compute_loss(X, y)

    # Itera sui pesi e calcola il gradiente numerico
    for param_name, param_value in model.params().items():
        # Salva il valore originale del parametro
        original_value = param_value.copy()

        # Perturba il parametro per calcolare la derivata
        param_value += epsilon
        loss_plus = model.compute_loss(X, y)

        param_value -= 2 * epsilon
        loss_minus = model.compute_loss(X, y)

        # Calcola il gradiente numerico
        gradients[param_name] = (loss_plus - loss_minus) / (2 * epsilon)

        # Ripristina il parametro
        param_value = original_value

    return gradients


def check_gradients(model, X, y, epsilon=1e-3):
    """
    Confronta i gradienti calcolati dalla backpropagation con quelli
    ottenuti tramite differenze finite.

    model: il modello della rete neurale
    X: dati di input
    y: etichette vere
    epsilon: piccolo valore per le differenze finite
    """
    # Calcola i gradienti tramite backpropagation
    backprop_gradients = model.backward(X, y)

    # Calcola i gradienti numerici
    numerical_gradients = numerical_gradient(X, y, model, epsilon)

    # Confronta i gradienti
    for param in model.params():
        numerical = numerical_gradients[param]
        backprop = backprop_gradients[param]

        error = np.abs(numerical - backprop) / (np.maximum(np.abs(numerical), np.abs(backprop)) + 1e-8)

        # Stampa l'errore relativo
        if (error > 1e-6).all(): # Se almeno uno degli errori è maggiore della soglia
            print(f"Errore sui gradienti per {param}: {error}")
        else:
            print(f"Gradienti per {param} passano il test.")



# testiamo la rete

# Parametri della rete
input_size = 3136
hidden_size = 1000
output_size = 1
learning_rate = 0.005

# Crea la rete neurale
nn = SimpleNN(input_size, hidden_size, output_size, learning_rate)

# carichiamo gli input
#true_vectors_path = "/content/MyAKarpathyMicrogradTest/signatures_real/dataset_sigle_real_vectors.npy"
#false_vectors_path = "/content/MyAKarpathyMicrogradTest/signatures_false/dataset_sigle_false_vectors.npy"


train_data, test_data, train_labels, test_labels = input_set(true_vectors_path, false_vectors_path)

# Allenare il modello
st = time.time()
nn.train(train_data, train_labels, epochs=1000)
et = time.time() -st
print(f"Training effettuato in {et:.4f} sec con {train_data.shape[0]} records")
# Testa un batch
st = time.time()
test_batch = test_data  # Prendi i primi i campioni di test
predictions = nn.forward(test_batch)
et = time.time() -st
print(f"Elapsed Time in sec: {et:.4f} for {test_batch.shape[0]} test records, equal to {et/test_batch.shape[0]:.4f} secs per record")
print(f"Predictions shape: {predictions.shape}")
#print(f"Predictions: {predictions}")


True vectors shape: (438, 3136)
False vectors shape: (528, 3136)
Dataset pronto: (966, 3136), Labels: (966, 1)
Train set> data: (772, 3136), labels: (772, 1), True: 363, False: 409 
Test set> data: (194, 3136), labels: (194, 1), True: 75, False: 119 
Epoch 0/1000, Loss: 1.5250749910461123
Epoch 100/1000, Loss: 0.6020341473806371
Epoch 200/1000, Loss: 0.5589006290474643
Reshuffling data at epoch 200
Epoch 300/1000, Loss: 0.5282464194969146
Epoch 400/1000, Loss: 0.5030221804033114
Reshuffling data at epoch 400
Epoch 500/1000, Loss: 0.48189907422823103
Epoch 600/1000, Loss: 0.46344377563598965
Reshuffling data at epoch 600
Epoch 700/1000, Loss: 0.44698780251628245
Epoch 800/1000, Loss: 0.4321171487346976
Reshuffling data at epoch 800
Epoch 900/1000, Loss: 0.41853663332536656
Training effettuato in 390.4928 sec con 772 records
Elapsed Time in sec: 0.0487 for 194 test records, equal to 0.0003 secs per record
Predictions shape: (194, 1)
